**Kaggle team name**: 🐧5-BestPicture-Cahill_Rivas 
1. Format: `🐧[BB Team#]-[Your fancy Kaggle group name]-[Students' names]`. 
  1. Eg. `🐧A-Heros-Fleischer,Melnikov`, where 🐧 identifies JHU and `A` identifies the Canvas group code

Your private LB score must be reproducible with this Colab. Seed all [RNG](https://en.wikipedia.org/wiki/Random_number_generation). Don't exceed runtime quota.

<small><hr style="margin:0;background-color:silver"><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a>, ©<a href="https://apps.ep.jhu.edu/course-homepages/3765-605-742-deep-neural-networks" target="_blank">JHU</a> 2021 onwards</font></small>

<hr color=darkblue>

# **🏆🍿Netflix**

<details><summary><font color=darkblue>More info and Kaggle API instructions</font></summary>

[Kaggle competition URL](https://www.kaggle.com/c/21422netflix/rules). See competition rules, submission, grading, dataset, and performance metric. The **starter code** below produces a baseline model, which you should beat, while respecting the competition rules. Your code starts after the timer. This is your baseline model. Seed all [RNG](https://en.wikipedia.org/wiki/Random_number_generation) for reproducibility!

**Instructions for enabling Kaggle API in Colab**:
1. Accept competition rules before running [Kaggle API](https://github.com/Kaggle/kaggle-api#api-credentials). [Loading Kaggle dataset example](https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab)
1. In your Kaggle Account, [Create API Token](https://github.com/Kaggle/kaggle-api#api-credentials) and save the resulting **kaggle.json** file to the [root of your Google Drive](https://drive.google.com/drive/u/0/my-drive) 
2. In Colab, open **Files** panel 🗀 (on the left) and click gray folder icon <font color=gray>🖿</font> to mount your Google drive

Your Kaggle/Google Drive credentials are secure; and Colab's kaggle.json only lasts a Colab session.

</details>


In [ ]:
# mount google drive
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if kaggle.json is stored in Google Drive

# special code to get my drive working (comment out)
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q --upgrade imgaug==0.2.5 > log.txt  # avoids warning/conflict with scipy upgrade
!pip install -q --upgrade scipy >>log.txt  # upgrades 1.4 to 1.8

In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 21422netflix             # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

- competition is now set to: 21422netflix
caution: filename not matched:  2722phonemes.zip
caution: filename not matched:  jan-30-22stellar.zip
Using competition: 21422netflix
 teamId  teamName                                                  submissionDate       score    
-------  --------------------------------------------------------  -------------------  -------  
8226841  DT-Make Peace!-Veips,Malchenko,Markevich                  2022-02-27 20:47:24  0.42480  
8192932  DA-Steve&TriDimona-Parakal, Kolobaev, Ignatyev, Kurbatov  2022-02-27 19:11:50  0.42360  
8227002  DQ-WeAreAllBrothers-Vasilev, Stepin, Repin                2022-02-27 20:39:18  0.42300  
8227620  DH-KMO-Korzun,Mozharov,Omarova                            2022-02-28 00:15:40  0.42300  
8228169  DI-MakeLove-Dzhkha,Alekseev,Ivanov                        2022-02-28 20:56:17  0.42300  
8224840  DM-Peace-Xu,Ivanova, Babakekhyan                          2022-02-27 20:20:54  0.42280  
8830908  Samuel Nathanson               

In [ ]:
%%time
%%capture
%reset -f

from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all" 
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage, plotly.express as px
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import TruncatedSVD
from scipy.sparse.linalg import svds
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=1000, precision=2, edgeitems=20, suppress=True)
pd.set_option('max_columns', 20, 'precision', 2, 'display.max_rows', 4)

CPU times: user 1.57 s, sys: 180 ms, total: 1.75 s
Wall time: 1.98 s


The matrix below contains ratings that users (user id or `uid`) have given to the movies (movie ID or `mid`). It is 85% blank, but the original dataset was 99% sparse (with ~1 rating per user) [&#127910;](https://youtu.be/ImpV70uLxyw). Your task is not to fill the whole matrix, but to impute `NaN` values in `XY_Netflix.csv`, which is an equivalent representation of `XY_Netflix_matrix.csv`. This is a collaborative filtering problem, where observations and features have some implicit relation. Recall that typically we expect independent observations in a sample (unless we have a time series or video of image frames, etc.). Here our model needs to learn and leverage the dependencies from observed movie ratings. Clustering techniques might be more intuitive here. [Truncated SVD](https://scikit-learn.org/stable/modules/decomposition.html#lsa) is also popular. Supervised models can be used as well, if we can build features describing our users (which we would treat as observations). Note that we have no other information on users except what they have ranked (which might suggest the cluster they belong to, which might offer unobserved (latent) features). [&#127910;](https://youtu.be/8wLKuscyO9I) Since users relate to other users through the movies they ranked, this can also be presented as a [bipartite graph](https://en.wikipedia.org/wiki/Bipartite_graph).

In [ ]:
dfMtx = pd.read_csv('XY_Netflix_matrix.csv', low_memory=False); dfMtx

,1,2,3,4,5,6,7,8,9,10,...,371,372,373,374,375,376,377,378,379,380
0,NaN,NaN,2.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,NaN,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128698,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
f = px.imshow(dfMtx.iloc[:2000,:].T, color_continuous_scale='RdBu_r', aspect='auto');  # movies by users (transposed for visual convenience)
f.update_layout(margin=dict(l=0,r=0,b=0,t=0))

In [ ]:
dfMtx.info()   # note the datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128699 entries, 0 to 128698
Columns: 380 entries, 1 to 380
dtypes: float64(380)
memory usage: 373.1 MB


The dataframe below is another representation of the matrix form above. It lists the `uid`$\times$`mid`locations of the values you need to predict (those with `NaN`) and all other non-zero values in the matrix above.

In [ ]:
df = pd.read_csv('XY_Netflix.csv', low_memory=False); df  # user ID, movie ID, rating. NaN=requires predictions

,uid,mid,rtg
0,1,1,NaN
1,2,2,NaN
...,...,...,...
7402070,49692,126,4.0
7402071,9783,37,1.0


In [ ]:
df.rtg.fillna(0).astype(int).value_counts().to_frame().T   # rating frequencies. 0 represents NaN

,4,3,5,2,1,0
rtg,2451087,2120585,1628122,842056,340222,20000


In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

A few thoughts:

1. First off, we loaded the movies.csv data and explored what was there. This was data that could help with more subtle clustering. In the end, the elements we chose were the result of many attempts at trying other approaches than the provided singular value decomp, the matrix factorization recommendation system strategy. The ratings values were ultimately considered sufficient for the evaluation of the predictions using this factorization approach. The online literature prefers to explain the factorization approach in terms of an adjacency list, which is based on two 3 variables with the ratings variable populating each cell of the prediction matrix or adjacency list.

2. We looked at the polarizing examples with more 1s, 2s and 5s. For example, Napoleon Dynamite had more 5s than any other ranking! We didn't end up using a special clustering or factor for polarizing movies, but if we had more time, we would have explored this. We understood that we wouldn't be evaluated based on continuous regression error but the easiest and most accessible method for use was the accuracy_score(), which we used for all model approaches while realizing that SVD is both a more interesting and accurate model approach for this data set.

3. We focused most of our time on trying to get the models working, which was the most interesting part of this project (for us). Given that, we didn't focus on trying to create new variables. In particular, a portion of our time was spent understanding the model approach in the starter notebook. It is a different style and strategy than we have previously experienced both professionally and academically. We weren't sure initially, if we could just do a substitition on the svds model since it was iterated over. After realizing this fact, it became necessary to research the model and check for alternative approaches to using SVD that would compare with our previous knowledge of modeling in the course. After exhausting our research exploration, we decided that the given df was sufficient to begin modeling, though it only had two variables, which is not the best feature space for prediction. We didn't do much feature engineering but we some quick fix feature generate such as creating dummies and filtering the initial feature space after adding the movies.csv data. 

In [ ]:
#load movies data
mdf = pd.read_csv('movies.csv'); mdf  # movie data

,mid,Year,Name,voteAvg,voteCnt,popularity,duration,income,budget,releaseDate,...,origLang,spokenLang,ttl,origTtl,URL,kwds,desc,prodComp,prodCtry,tagline
0,1,2004,The Village,6.2,1071,27.49,108.0,256697520,60000000,7/30/04,...,en,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Village,The Village,NaN,"[{""id"": 1328, ""name"": ""secret""}, {""id"": 5774, ...",When a willful young man tries to venture beyo...,"[{""name"": ""Scott Rudin Productions"", ""id"": 258...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",There is no turning back
1,2,1997,Liar Liar,6.4,1424,26.72,86.0,181000000,45000000,3/21/97,...,en,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Liar Liar,Liar Liar,NaN,"[{""id"": 387, ""name"": ""california""}, {""id"": 102...",Fletcher Reede is a fast-talking attorney and ...,"[{""name"": ""Imagine Entertainment"", ""id"": 23}, ...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",Trust me.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,379,1999,The Big Tease,5.8,6,0.50,87.0,0,0,1/1/99,...,en,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Big Tease,The Big Tease,thebigtease.warnerbros.com/index.html,"[{""id"": 928, ""name"": ""hairdresser""}, {""id"": 19...",Thinking he's competing in Los Angeles' hot Pl...,[],[],He saw. He combed. He conquered.
379,380,2004,Tae Guk Gi: The Brotherhood of War,7.4,133,9.57,140.0,15,0,2/5/04,...,ko,"[{""iso_639_1"": ""ko"", ""name"": ""\ud55c\uad6d\uc5...",Tae Guk Gi: The Brotherhood of War,태극기 휘날리며,NaN,"[{""id"": 41586, ""name"": ""archaeologist""}, {""id""...","In 1950, in South Korea, shoe-shiner Jin-tae L...","[{""name"": ""Showbox Entertainment"", ""id"": 11420}]","[{""iso_3166_1"": ""KR"", ""name"": ""South Korea""}]",NaN


## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

This was a fascinating and difficult one. As we learned in the video about the team that won the actual Netflix $1M prize, they spent over a year on it and it took many different models & approaches to keep making progress. We feel the same way. 

Here's what we did and why, including some things we never quite got to work in the time allotted:

1. Checked to make sure mean, median or mode wasn't any better than the base SVD model. It wasn't. SVD won. 

2. Tuned SVD hyperparameters. We noticed k=10 was hard-coded in. We explored a range of different numbers from 5-20. We found that k=9 delivered a slight improvement to the uncoditional distribution chart, as well as the baseline score in Kaggle.

3. Explored other matrix demoposition techniques. They were promising, but we couldn't get them to deliver a better score. With more time, we would further explore these.

4. K-Means clustering. We were excited to try this (at least one of us had never used it before). We got it working, but silhouette score was low. It still didn't beat svd. If we had more time, we might have found a way to use it. We also didn't have time to use movie-specific data to improve clustering. That likely would have helped. 

5. Explored using Random Forest combined with the movie data. This seemed to perform well in accuracy using exiting data as test, but we had a hard time getting it to work. We think our indexes might have been off in the submission, and didn't have time to fix. With more time, we would explore this and possibly combine it into a ranked voting system.

6. Finding ways to better match the actual probability distribution. SVD and many other models overstate 3 and 4 rankings and understate 1, 2, and 5 rankings (e.g. the Napoleon Dynamite challenge). We tried testing a brute-force version of this where any movie that had a mode of 1, 2, or 5, we hard-coded the mode as rating for any NaN value. That slightly reduced performance. We tried only doing it for the less likely 1 or 2 modes, which slightly increased performance. We would be interested in exploring this better, e.g. creating a special clustering approach for polarizing movies.

7. We might have got to it late but we were able to at least gather and structure the code to do random forest modeling on the data. We chose random forest because we have both experienced it's performance benefits on various kinds of data among the wide variety of ensemble methods. 

8. Because the issue of this data set really about two very important things, we wanted to do our best, when we time permitted, to focus our efforts on choosing the best features among those in the provide movies.csv file. This file contains a lot essential but since time is limited to only a week, we could explore the natural language processing on the more text based columns.  

9. We were both very happy to have been provided this data set as it gave us chance to think about SVD and it's application to modeling in the realm of a limited feature space as well as the less f matrix factorization. We will be more open to considering these types of techniques in the future after working on this notebook. 

10. We completed the random forest model and using accuracy_score() was an interesting experience. It gave the starter notebook model an accuracy of 0.304 and the rf model an accuracy_score() of 0.379 but the kaggle score was 0 on both trial submissions. Altogether, the experience of comparison was a great task and a difficult one. Figuring out the indexing was a matter of interest and we believe that was the root issue. A link to rf model is provided: https://colab.research.google.com/drive/1OUsR7ykIHGcIWRTy5eCrYidU5vsAGNxx?usp=sharing


All in all, we enjoyed having such a challenging assignment. We learned about some new tools (e.g. matrix factorization). If we had a longer time period, we could have pursued many of these, possibly in combination with each other.

Below is a baseline model that produces the baseline result on Kaggle leaderboard (LB).

In [ ]:
if 'dfMtx' in vars(): del dfMtx   # release ~400MB RAM
dfK = df[df.rtg!=df.rtg]          # uid-mid-rtg for Kaggle submission (contains NaN ratings)
dfX = df[df.uid.isin(dfK.uid.unique())]  # all ratings on users requiring predictions (as adjacency list)
dfX = dfX.pivot_table(index='uid', columns='mid', values='rtg')  # convert to (adjacency) matrix format
dfX_est = dfX.fillna(dfX.mean())  # fill NaN with movies' mode ratings, save as
n, p = dfX.shape                  # number of users and movies

In [ ]:
dfX.iloc[:2,:30]   # original matrix with NaN's

mid,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
uid,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN
2,5.0,NaN,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,3.0,NaN,NaN,3.0,NaN


In [ ]:
dfX_est.loc[:2,:30]  # naive imputation with column means

mid,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
uid,,,,,,,,,,,,,,,,,,,,,
1,3.27,3.52,2.0,3.69,3.29,3.63,5.00,3.53,2.78,3.37,...,3.88,3.04,3.0,3.22,3.52,5.0,3.06,3.12,3.37,3.53
2,5.00,3.52,5.0,3.00,3.29,3.63,4.26,3.53,2.78,3.37,...,3.88,3.04,5.0,3.22,3.52,3.0,3.06,3.12,3.00,3.53


In [ ]:
np.random.seed(0)
for _ in range(5):                             # apply truncated SVD approximation a few times
  U, D, Vt = svds(dfX_est, k=9)               # find k eigenvectors (U,V) and eigenvalues (D)
  dfX_est.loc[1:n, 1:p] = U @ np.diag(D) @ Vt  # k-dimensional estimate (as np array) of dfM (we expect ratings' noise to reduce)
  
  # to handle extreme ratings:
  # for movies with mode of 1 or 2, assign them that rating
  for m in range(1, dfX.shape[1]):
    # if dfX[m].mode(dropna=True).values[0] == 5: # performance suffered when we hard-coded 5
    #   dfX_est[m] = 5
    if dfX[m].mode(dropna=True).values[0] == 2:
      dfX_est[m] = 2
    if dfX[m].mode(dropna=True).values[0] == 1:
      dfX_est[m] = 1
  mask = ~np.isnan(dfX)       # mask locations of NaN's
  dfX_est[mask] = dfX[mask]   # place original/known ratings back into estimated uid×mid matrix 
dfX_est.loc[:2,:30]           # a "better" approximation of underlying ratings distribution (or so we hope ;) )

mid,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
uid,,,,,,,,,,,,,,,,,,,,,
1,2.39,3.76,2.0,3.02,3.22,3.23,5.00,3.72,2.48,2.99,...,4.65,2.89,3.0,3.28,3.48,5.0,2.85,2.63,3.28,3.50
2,5.00,4.30,5.0,3.00,3.21,4.20,4.26,4.32,3.22,3.88,...,3.72,3.20,5.0,3.28,3.56,3.0,3.39,3.36,3.00,3.42


In [ ]:
# #####################
# # K-Means clustering –– didn't improve score, commented out
# #####################
# from sklearn.cluster import KMeans
# k=10  # tested k=3 to k=100 using inertia, silhouette score and rating distribution vs. svd

# # assign every user id to one of k clusters
# kmeans = KMeans(n_clusters=k)
# clusters = kmeans.fit_predict(dfX_est.values)

# # create new dataframe with cluster index
# dfX_kmeans = dfX_est.copy(deep=True)  # building on SVD (also tried using mean imputation)
# dfX_kmeans['KCluster'] = clusters

# # identify means for each cluster
# cluster_means = dfX_kmeans.groupby(['KCluster']).mean()

# # drop KCluster column
# dfX_kmeans = dfX_kmeans.drop(columns=['KCluster']) 

# # fill every row with cluster means
# for i in range(len(dfX_kmeans)):
#   dfX_kmeans.iloc[i] = cluster_means.iloc[clusters[i]]

# # fill back in known values
# dfX_kmeans[mask] = dfX[mask]   # place original/known ratings back into estimated uid×mid matrix 
# dfX_kmeans.loc[:2,:30]           # kmeans estimations, building on SVD
# print("inertia:", kmeans.inertia_)

# # evaluate silhouette score for KMeans (tested values from k=3 to k=100 –– all were poor)
# from sklearn.metrics import silhouette_score
# silhouette_score(dfX_est, kmeans.labels_)

Generate submission for SVD

In [ ]:
# Submissions are scored on accuracy, not on continuous/regression error
dfX_est_L = dfX_est.unstack().reset_index().round(0).astype(int).rename(columns={0:'rtg'})  # tall format with integer ratings
dfK = dfX_est_L.merge(dfK, on=['mid','uid'], suffixes=['','_']).drop('rtg_', axis=1)
dfK.index = (dfK.uid-1)*dfK.mid.max()+dfK.mid
dfK['rtg'] = dfK['rtg'].clip(1, 5)  # restrict the range to 1,2,3,4,5 . Thanks to Yannis & Liz, See Piazza @67_f1
dfK

In [ ]:
# evaluate unconditional distributions of observed and predicted ratings. Learn from the differences to focus on wrong predictions
plt.subplot(1, 1, 1)
_ = df.rtg.dropna(axis=0).astype(int).hist(density=True, alpha=0.3);
_ = dfK.rtg.astype(int).hist(density=True, alpha=0.3);
_ = plt.legend(['actual','predicted']);

In [ ]:
# ### KMEANs
# # Submissions are scored on accuracy, not on continuous/regression error
# dfX_est_L = dfX_kmeans.unstack().reset_index().round(0).astype(int).rename(columns={0:'rtg'})  # tall format with integer ratings
# dfK = dfX_est_L.merge(dfK, on=['mid','uid'], suffixes=['','_']).drop('rtg_', axis=1)
# dfK.index = (dfK.uid-1)*dfK.mid.max()+dfK.mid
# dfK['rtg'] = dfK['rtg'].clip(1, 5)  # restrict the range to 1,2,3,4,5 . Thanks to Yannis & Liz, See Piazza @67_f1
# dfK

In [ ]:
# # KMEANS
# # evaluate unconditional distributions of observed and predicted ratings. Learn from the differences to focus on wrong predictions
# plt.subplot(1, 1, 1)
# _ = df.rtg.dropna(axis=0).astype(int).hist(density=True, alpha=0.3);
# _ = dfK.rtg.astype(int).hist(density=True, alpha=0.3);
# _ = plt.legend(['actual','predicted']);

In [ ]:
ToCSV(dfK[['rtg']].sort_index(), 'MySubmission')

# **References:**

1. Aurélien Géron. Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems, 2nd Edition. 2019. Focused on K-Means section and implementation in Chapter 9.

2. "Collaborative Filtering: Matrix Factorization Recommender System." Jiri Stodulka. https://www.jiristodulka.com/post/recsys_cf/. Accessed June 30, 2022.

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

## 💡**Starter Ideas**

1. Tune model **hyperparameters**. 
  1. SVD's $k$ might be user-perceived movie genres (perhaps, an average user categorizes movies into just 5 or 50 genres) [🎦](https://youtu.be/sooj-_bXWgk)
1. Try smarter **sampling** from the training set to avoid using all observations (due to time constraint)
  1. Some movies may have consistent high/low ratings and do not distinguish user preferences (just add to runtime)
1. Try smarter initial imputation, perhaps, conditioned on movie genres or other observed features
1. Try clustering techniques from [SKL](https://scikit-learn.org/stable/modules/clustering.html)
1. Try engineering new features using Movies' data (`movies.csv`)
1. Try supervised approach on clusters or engineered features
1. Try shift/scale ratings. Recall: SVD seeks largest quadratic error from mean without shifting first, like variance does. Since $1^2\ll5^2$, SVD might work better for higher ratings.
  1. Note: the baseline model squeezes predictions towards ratings 3 and 4, which is not the original distribution.
1. Try feature restriction, selection, and regularization (see ESL pp.140-141)
1. Do a thorough EDA. Are there movies that are liked by most viewers?
1. Try Robust PCA (RPC) [🎦](https://www.youtube.com/watch?v=yDpz0PqULXQ)
1. Find polarized movies and tune for them (like "*Napoleon Dynamite*"). See [&#127910;](https://youtu.be/ImpV70uLxyw)
1. Try KNN [📄](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.143.596&rep=rep1&type=pdf), [📄](https://scholar.google.com/scholar?q=knn+netflix+prize)
1. Learn more methods to build/use latent representations [🎦](https://youtu.be/n3RKsY2H-NE),  [📄](https://ieeexplore.ieee.org/abstract/document/5197422), [📄](https://scholar.google.com/scholar?q=netflix+prize+challenge), [&#127910;](https://www.youtube.com/results?search_query=netflix+prize+svd), [📄](https://www.aaai.org/Papers/AAAI/2002/AAAI02-029.pdf), 